In [1]:
#Import Libraries:
import os
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pickle

In [22]:
page = 1
house_infos = list()
while True:
    try:
        drive = webdriver.Chrome(ChromeDriverManager().install())
        drive.maximize_window()
        wait = WebDriverWait(drive, 10)
        drive.get('https://nha.chotot.com/thue-phong-tro-da-nang?page=' + str(page))
    except: break
    houses = drive.find_elements(By.XPATH, value = '//a[@href and @class="AdItem_adItem__gDDQT"]')
    if (houses == []): break
    for i in range(len(houses)):
        houses[i] = houses[i].get_attribute('href')
    drive.close()
    for house in houses:
        drive = webdriver.Chrome(ChromeDriverManager().install())
        drive.maximize_window()
        wait = WebDriverWait(drive, 10)
        drive.get(house)
        try:
            house_location= drive.find_element(By.XPATH, value = '//span[@class="fz13"]').text.replace('Xem bản đồ','')
            house_price = drive.find_element(By.XPATH, value = '//span[@itemprop="price"]').text
            house_features = drive.find_elements(By.XPATH, value = '//div[@class="col-xs-6 no-padding AdParam_adParamItemVeh__mCpPS" and @data-testid="param-item"]')
            house_info = {}
            house_info['Location'] = house_location
            house_info['Price'] = house_price
            for index in range(len(house_features)):
                if house_features[index].text[0] == 'T': house_info['Feature' + str(2)] = house_features[index].text
                elif house_features[index].text[0] == 'S': house_info['Feature' + str(3)] = house_features[index].text
                else: house_info['Feature' + str(index)] = house_features[index].text
            house_infos.append(house_info)
        except: continue
        drive.close()
    with open('save_page.txt','w') as file:
        file.write(str(page))
        file.close()
    if (page % 10 == 0):
        with open('../Raw_Data/chotot.pickle','wb') as handle:
            pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)
    page+=1
with open('../Raw_Data/chotot.pickle','wb') as handle:
    pickle.dump(house_infos, handle, protocol=pickle.HIGHEST_PROTOCOL)

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:01<00:00, 6.06MB/s]


KeyboardInterrupt: 

In [2]:
with open("../Raw_Data/chotot.pickle","rb") as handle:
    dataset = pickle.load(handle)
dataset = pd.DataFrame(data = dataset)
dataset = dataset[['Location', 'Price', 'Feature1']]
dataset = dataset.rename(columns = {'Feature1' : 'Area'})
dataset

,Location,Price,Area
0,"Đường Tôn Đản, Phường Thanh Khê Đông, Quận Tha...","2,3 triệu/tháng - 23 m2",Diện tích: 23 m²
1,"Đường Lê Hữu Trác, Phường Phước Mỹ, Quận Sơn T...",3 triệu/tháng - 25 m2,Diện tích: 25 m²
2,"Đinh Công Trứ, Phường Mỹ An, Quận Ngũ Hành Sơn...","1,6 triệu/tháng - 15 m2",Diện tích: 15 m²
3,"K814A/30/3, Trần Cao Vân, Phường Thanh Khê Đôn...","3,2 triệu/tháng - 30 m2",Diện tích: 30 m²
4,"Đường Số 3, Phường Hòa Thọ Tây, Quận Cẩm Lệ, Đ...",700.000 đ/tháng - 16 m2,Diện tích: 16 m²
...,...,...,...
620,"Đường Nguyễn Xí, Phường Hòa Minh, Quận Liên Ch...",7 triệu/tháng - 50 m2,Diện tích: 50 m²
621,"Đường Tôn Đản, Phường Thanh Khê Đông, Quận Tha...","2,3 triệu/tháng - 23 m2",Diện tích: 23 m²
622,"K 91/53, Đường Cù Chính Lan, Phường Hòa Khê, Q...","1,9 triệu/tháng - 25 m2",Diện tích: 25 m²
623,"02, Ngô Mây, Phường Hòa Xuân, Quận Cẩm Lệ, Đà ...","2,5 triệu/tháng - 40 m2",Diện tích: 40 m²


In [ ]:
dataset.to_csv('../Raw_Data/chotot.csv')